<a href="https://colab.research.google.com/github/fjprcosta/Projeto-Final-Soulcode-tema-sa-de/blob/main/Atvdesafiopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Conectores

In [ ]:
!pip install gcsfs

In [ ]:
from google.cloud import storage
import os
# lib para conexão
import requests
import numpy as np
# Modelagem de dados
import pandas as pd
# Ignorando avisos
import warnings
warnings.filterwarnings('ignore')
# URL da API da Alesp
URL_Despesas = 'http://www.al.sp.gov.br/repositorioDados/deputados/despesas_gabinetes.xml'
URL_Cadastro = 'https://www.al.sp.gov.br/repositorioDados/deputados/deputados.xml'

In [ ]:
#conectando drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
serviceAccount = '/content/drive/MyDrive/Datasets/dev-fiber-349020-e47e934c596f.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Pegando os dados
Conexao_01 = requests.get( URL_Despesas )
Conexao_02 = requests.get( URL_Cadastro )
# Verificando se a API funcinou
print( Conexao_01, Conexao_02 )

In [ ]:
#dataframe arquivo 1
df2 = pd.read_xml(URL_Cadastro, encoding='utf-8')

In [ ]:
#dataframe arquivo 2
df1 = pd.read_xml(URL_Despesas, encoding='utf-8')

##E --- Pré Análise

In [ ]:
#criando cópias dos df
despesas = df1.copy()
cadastro = df2.copy()

In [ ]:
despesas.head(1)

In [ ]:
#Trabalhando com os dados dos 4 últimos anos
dfanos = despesas.loc[(despesas['Ano'] >= 2019) & (despesas['Ano'] < 2023)]

In [ ]:
#agrupando por ano
dfanos.groupby(['Ano']).Valor.sum()

In [ ]:
#verificando se CNPJ tem somente valores únicos
dfanos.CNPJ.is_unique

In [ ]:
#verificando se Matricula tem somente valores únicos
dfanos.Matricula.is_unique

In [ ]:
#procurando inconsistências
sorted(pd.unique(dfanos['Fornecedor']))

In [ ]:
#tratando inconsistências
dfanos.replace(regex=[r'PEDAGIO','PEDÁGIOS','PEDAGIOS','PEGAGIO','CUPONS DE PEDÁGIO'],value='PEDÁGIO', inplace=True)

In [ ]:
#tratando inconsistências
dfanos.replace(regex=[r'DELTA RADIO TAXI','taxi'],value='TAXI', inplace=True)

In [ ]:
#contando campos nulos
dfanos.isna().sum()

In [ ]:
#completando valores nulos
despesas_tratadas = dfanos.fillna('Na')

In [ ]:
#Transformando CNPJ em FLOAT
dfanos["CNPJ"] = pd.to_numeric(dfanos['CNPJ'],errors="coerce")

In [ ]:
#confirmando as alterações no nulos
despesas_tratadas.isna().sum()

In [ ]:
#procurando inconsistências
pd.unique(cadastro['Telefone'])
#coluna Aniverário com null/null

##T --- Tratamento dos Dados

In [ ]:
#maiores despesas nos últimos 4 anos
despesas_tratadas.groupby(['CNPJ','Fornecedor','Tipo']).Valor.sum().sort_values(ascending=False).head()

CNPJ              Fornecedor                         Tipo                                                            
2491558000142.0   UNIDAS VEÍCULOS ESPECIAIS S.A.     O - LOCAÇÃO DE VEÍCULO                                              1883085.97
58488099000100.0  NEVADA RENT A CAR LTDA             O - LOCAÇÃO DE VEÍCULO                                              1606952.52
3067942000185.0   NOGUEIRA E NOGUEIRA JUNIOR LTDA    O - LOCAÇÃO DE VEÍCULO                                               784673.50
18870651000106.0  QUATTROPORTO CONS E NEGOCIOS LTDA  O - LOCAÇÃO DE VEÍCULO                                               568722.33
2558157000162.0   TELEFONICA BRASIL S.A              H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)     479502.07
Name: Valor, dtype: float64

In [ ]:
#3 deputados que mais gastaram
despesas_tratadas.groupby(['Deputado']).Valor.sum().sort_values(ascending=False).head(3)

Deputado
ENIO TATTO            1180638.64
TEONILIO BARBA        1169566.84
VINÍCIUS CAMARINHA    1164305.11
Name: Valor, dtype: float64

In [ ]:
#3 deputados que menos gastaram
despesas_tratadas.groupby(['Deputado']).Valor.sum().sort_values().head(3)

Deputado
PEDRO KAKÁ             1302.67
CARLOS BEZERRA JR.     1467.37
LUIZ GONZAGA VIEIRA    2251.96
Name: Valor, dtype: float64

In [ ]:
#quando se agrupa os campos, há uma divergencia de quem gastou mais. Sendo identificado uma inconsistencia
despesas_tratadas.groupby(['Deputado','CNPJ','Fornecedor','Tipo']).Valor.sum().sort_values(ascending=False).head()

Deputado          CNPJ             Fornecedor                               Tipo                                                            
MARCIO NAKASHIMA  9167845843.0     MARCOS CARVALHO ELIAS                    L - LOCAÇÃO DE BENS IMÓVEIS                                         463628.26
TEONILIO BARBA    10778146820.0    MARCELO PEREIRA NUNES                    L - LOCAÇÃO DE BENS IMÓVEIS                                         364251.17
TENENTE COIMBRA   16228518895.0    SILVIO CESAR DE JESUS SANTOS             L - LOCAÇÃO DE BENS IMÓVEIS                                         288692.54
ROBERTO MORAIS    51410672891.0    JOSÉ CARLOS RIBEIRO                      L - LOCAÇÃO DE BENS IMÓVEIS                                         278869.00
LECI BRANDÃO      8300679000146.0  CIRCUS FILMES E PRODUÇOES GRAFICAS LTDA  F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)    277200.00
Name: Valor, dtype: float64

In [ ]:
#plotando os os valores maiores
despesas_tratadas.groupby(['CNPJ','Tipo']).Valor.sum().sort_values(ascending=False).head(3).plot.bar(figsize=(10,8),xlabel="TIPO DE GASTO",ylabel="VALOR")

In [ ]:
#plotando os 3 deputados que menos gastaram
despesas_tratadas.groupby(['Deputado']).Valor.sum().sort_values().head(3).plot.bar(figsize=(10,8),xlabel="TIPO DE GASTO",ylabel="VALOR")

In [ ]:
#plotando os 3 deputados que mais gastaram
despesas_tratadas.groupby(['Deputado']).Valor.sum().sort_values(ascending=False).head(3).plot.bar(figsize=(10,8),xlabel="TIPO DE GASTO",ylabel="VALOR")

##L --- Salvando e fazendo upload para o GCP

In [ ]:
#CÓDIGO QUE ACESSA A BUCKET CRIADA E FAZ O DOWNLOAD DOS ARQUIVOS VIA PANDAS
client = storage.Client()
#CRIAR UMA VARIÁVEL CHAMADA BUCKET QUE VAI RECEBER O NOME DA BUCKET DO CLOUD STORAGE
bucket = client.get_bucket('felipesoulcode')

In [ ]:
despesas_tratadas.to_csv('gs://felipesoulcode/tratados/despesas-tratadas.csv')